In [4]:
import os
import pygame
from gtts import gTTS
from time import sleep


pygame 2.6.0 (SDL 2.28.4, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
def speak_response(response_text):
    tts = gTTS(text=response_text, lang='en')
    audio_file = "response.mp3"
    tts.save(audio_file)

    pygame.mixer.init()
    pygame.mixer.music.load(audio_file)
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        sleep(0.1)

    pygame.mixer.music.stop()
    pygame.mixer.quit()

    sleep(0.5)

    if os.path.exists(audio_file):
        for _ in range(3):
            try:
                os.remove(audio_file)
                break
            except PermissionError:
                print("Could not delete file, retrying...")
                sleep(1)

In [6]:
import speech_recognition as sr

def capture_speech(mic_index):
    recognizer = sr.Recognizer()
    mic = sr.Microphone(device_index=mic_index)
    
    with mic as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    
    try:
        print("Recognizing speech...")
        user_input = recognizer.recognize_google(audio)
        print(f"User said: {user_input}")
        return user_input
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None
    except sr.RequestError as e:
        print(f"Error with the Google Speech Recognition service; {e}")
        return None


In [7]:
def list_microphones():
    mic_list = sr.Microphone.list_microphone_names()
    print("Available microphones:")
    for i, mic_name in enumerate(mic_list):
        print(f"Index {i}: {mic_name}")
    return mic_list


In [8]:
def select_microphone():
    mic_list = list_microphones()
    index = 2  # Automatically selecting index 2 microphone
    print(f"Selected microphone: {mic_list[index]}")
    return index


In [9]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

# Track conversational context
conversation_history = []

def generate_response(user_input):
    model_name = "facebook/blenderbot-3B"  # Switched to a larger, more capable model
    tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
    model = BlenderbotForConditionalGeneration.from_pretrained(model_name)
    
    # Add user input to conversation history
    conversation_history.append(user_input)
    
    # Join conversation history to provide context
    conversation_context = " ".join(conversation_history[-5:])  # Keeping the last 5 exchanges
    
    # Adjust parameters for better logic and factual accuracy
    inputs = tokenizer(conversation_context, return_tensors="pt", truncation=True, max_length=128)
    reply_ids = model.generate(
        inputs['input_ids'],
        max_length=512, 
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.8,  # Lower temperature for more logical responses
        num_beams=5,  # Beam search for more precise responses
        no_repeat_ngram_size=2  # Avoid repetition
    )
    
    bot_response = tokenizer.decode(reply_ids[0], skip_special_tokens=True)
    
    # Add bot response to the conversation history
    conversation_history.append(bot_response)
    
    return bot_response


In [10]:
def amigo_conversational_companion():
    # Step 1: Greeting
    greeting = "Hello, I am Amigo. How can I help you today?"
    print(f"Amigo: {greeting}")
    speak_response(greeting)
    
    # Step 2: Select microphone
    mic_index = select_microphone()
    
    while True:
        # Step 3: Capture user speech
        user_input = capture_speech(mic_index)
        
        if user_input is None:
            continue
        
        # Step 4: Generate bot response with context and factual accuracy
        bot_response = generate_response(user_input)
        print(f"Amigo: {bot_response}")
        
        # Step 5: Speak the response
        speak_response(bot_response)


In [ ]:
# Start the conversation
amigo_conversational_companion()


Amigo: Hello, I am Amigo. How can I help you today?
Available microphones:
Index 0: Microsoft Sound Mapper - Input
Index 1: Stereo Mix (Realtek(R) Audio)
Index 2: Microphone (Realtek(R) Audio)
Index 3: Microsoft Sound Mapper - Output
Index 4: Speaker / Headphone (Realtek(R)
Index 5: Primary Sound Capture Driver
Index 6: Stereo Mix (Realtek(R) Audio)
Index 7: Microphone (Realtek(R) Audio)
Index 8: Primary Sound Driver
Index 9: Speaker / Headphone (Realtek(R) Audio)
Index 10: Speaker / Headphone (Realtek(R) Audio)
Index 11: Stereo Mix (Realtek(R) Audio)
Index 12: Microphone (Realtek(R) Audio)
Index 13: Microphone (Realtek HD Audio Mic input)
Index 14: Stereo Mix (Realtek HD Audio Stereo input)
Index 15: Speakers (Realtek HD Audio output)
Index 16: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(Airdopes Unity ANC))
Index 17: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free AG Audio%0
;(Airdopes Unity ANC))
Index 18: Headphones ()
Index 19: Headphones ()
In